## Settings

In [1]:
# 一つ上の階層からモジュールを参照できるようにする
import sys
sys.path.append('..')

In [2]:
# ロギング用
import os
from datetime import datetime, timezone, timedelta
from logs.logger import create_logger, get_logger
# モデル作成補助用
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from features.funcs import load_feather, load_feather_ohe
from evals.funcs import get_pred_result, get_acc_and_logloss, print_conf_matrix
from inputs.funcs import load_data
# モデル作成用
from models.keras import ModelKeras
from keras.layers import Dense, Dropout

In [3]:
# モジュールの変更を自動的に反映する
%load_ext autoreload
%autoreload 2

In [4]:
# loggingの設定を行う
today = datetime.now(timezone(timedelta(hours=9))).strftime('%Y%m%d')
exp_version = today
os.environ['exp_version'] = exp_version
create_logger(exp_version)

In [5]:
os.listdir('../features/nn')

['fare_test_std_te.feather',
 'age_train_std_te.feather',
 'fare_train_std_te.feather',
 'age_test_std_mean.feather',
 'test_ohe.feather',
 'train_ohe_2_z.feather',
 'age_train_std_mean.feather',
 'age_test_std_te.feather',
 'fare_test_std_mean.feather',
 'train_ohe.feather',
 'fare_train_std_mean.feather',
 'test_ohe_2_z.feather']

In [63]:
def save_submission(submission_df):
    file_name = 'keras_submission_{}'.format(today)
    file_name_suffix = input(f'Enter File Suffix: {file_name}_...')
    submission_file_path = f'submissions/nn/{file_name}_{file_name_suffix}.csv'
    submission_df.to_csv(submission_file_path, index=False)

## Keras

### Base

In [57]:
train_x, train_y, test_x = load_data()

In [58]:
train_x = train_x.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Sex', 'Embarked'], axis=1)
train_x = train_x.dropna()
train_y = train_y.iloc[train_x.index]

In [59]:
model = ModelKeras(logging=True)
layer1 = Dense(1024, activation='relu', input_shape=(train_x.shape[1],))
layer2 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer4 = Dense(1, activation='sigmoid')
model.add_layers(layer1, layer2, layer3, layer4)
model.compile()
model.set_fit_params(epochs=10, verbose=0)

[INFO] 2021-02-14 09:21:22,291 >>	=== NN KERAS MODEL ===


In [60]:
pred_train = get_pred_result(model, train_x, train_y)
get_acc_and_logloss(pred_train, train_y, logging=True)

[INFO] 2021-02-14 09:21:27,981 >>	ACCURACY: 0.6792717086834734
[INFO] 2021-02-14 09:21:27,982 >>	LOGLOSS: 0.6165512822206677
[INFO] 2021-02-14 09:21:27,983 >>	data size: 714
[INFO] 2021-02-14 09:21:27,983 >>	correct predictions: 485
[INFO] 2021-02-14 09:21:27,984 >>	


In [61]:
age_mean = train_x['Age'].mean()
fare_mean = train_x['Fare'].mean()
test_x = test_x.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Sex', 'Embarked'], axis=1)
test_x.loc[test_x['Age'].isna(), 'Age'] = age_mean
test_x.loc[test_x['Fare'].isna(), 'Fare'] = fare_mean
preds = model.predict(test_x)
preds = [round(pred[0]) for pred in preds]

In [68]:
submission = pd.DataFrame({'PassengerId': passenger_id, 'Survived': preds})
save_submission(submission)

Enter File Suffix: keras_submission_20210214_... 1


#### Age standardized and target encoding

In [69]:
train_x, train_y, test_x = load_data()

In [70]:
base_path = '../features/nn/'
cols_to_rplc_train = {
    'Age': base_path + 'age_train_std_te.feather',
    'Fare': base_path + 'fare_train_std_te.feather'
}
train_x = load_feather(train_x, cols_to_rplc_train)
train_x = load_feather_ohe(train_x, base_path + 'train_ohe.feather')
train_x = load_feather_ohe(train_x, base_path + 'train_ohe_2_z.feather')

In [71]:
train_x = train_x.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Sex', 'Embarked', 'Pclass'], axis=1)
train_x = train_x.drop([col for col in train_x.columns if 'Cabin' in col ], axis=1)
train_x = train_x.drop([col for col in train_x.columns if 'Ticket' in col ], axis=1)

In [72]:
model = ModelKeras(logging=True)
layer1 = Dense(1024, activation='relu', input_shape=(train_x.shape[1],))
layer2 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer4 = Dense(1, activation='sigmoid')
model.add_layers(layer1, layer2, layer3, layer4)
model.compile()
model.set_fit_params(epochs=10, verbose=1)

[INFO] 2021-02-14 10:37:05,956 >>	=== NN KERAS MODEL ===


In [73]:
pred_train = get_pred_result(model, train_x, train_y)
get_acc_and_logloss(pred_train, train_y, logging=False)

Epoch 1/10
21/21 [==============================] - 0s 13ms/step - loss: 0.5051 - accuracy: 0.7605 - val_loss: 0.3859 - val_accuracy: 0.8206
Epoch 2/10
21/21 [==============================] - 0s 8ms/step - loss: 0.3937 - accuracy: 0.8278 - val_loss: 0.4185 - val_accuracy: 0.8206
Epoch 3/10
21/21 [==============================] - 0s 7ms/step - loss: 0.3988 - accuracy: 0.8308 - val_loss: 0.4102 - val_accuracy: 0.8161
Epoch 4/10
21/21 [==============================] - 0s 7ms/step - loss: 0.3905 - accuracy: 0.8338 - val_loss: 0.3918 - val_accuracy: 0.8161
Epoch 5/10
21/21 [==============================] - 0s 8ms/step - loss: 0.3850 - accuracy: 0.8443 - val_loss: 0.3804 - val_accuracy: 0.8251
Epoch 6/10
21/21 [==============================] - 0s 8ms/step - loss: 0.3922 - accuracy: 0.8353 - val_loss: 0.3633 - val_accuracy: 0.8206
Epoch 7/10
21/21 [==============================] - 0s 8ms/step - loss: 0.3773 - accuracy: 0.8398 - val_loss: 0.3751 - val_accuracy: 0.8251
Epoch 8/10
21/21 [=

In [27]:
model.fit(tr_x=train_x, tr_y=train_y, va_x=train_x, va_y=train_y)

In [28]:
passenger_id = test_x['PassengerId']

In [29]:
base_path = '../features/nn/'
cols_to_rplc_train = {
    'Age': base_path + 'age_test_std_te.feather',
    'Fare': base_path + 'fare_test_std_te.feather'
}
test_x = load_feather(test_x, cols_to_rplc_train)
test_x = load_feather_ohe(test_x, base_path + 'test_ohe.feather')
test_x = load_feather_ohe(test_x, base_path + 'test_ohe_2_z.feather')

In [30]:
test_x = test_x.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Sex', 'Embarked', 'Pclass'], axis=1)
test_x = test_x.drop([col for col in test_x.columns if 'Cabin' in col], axis=1)
test_x = test_x.drop([col for col in test_x.columns if 'Ticket' in col], axis=1)

In [36]:
prediction = model.model.predict(test_x).ravel()
# prediction = [int(pred) for pred in prediction]

In [32]:
submission = pd.DataFrame({'PassengerId': passenger_id, 'Survived': prediction})

In [33]:
file_number = input('Enter File Number')
submission_file_path = 'submissions/keras_submission_{}_{}.csv'.format(today, file_number)
if file_number != 0:
    submission.to_csv(submission_file_path, index=False)

Enter File Number 0


#### Age standardized and mean

In [23]:
train_x, train_y, _ = load_data()

In [24]:
base_path = '../features/nn/'
cols_to_rplc_train = {
    'Age': base_path + 'age_train_std_mean.feather',
    'Fare': base_path + 'fare_train_std_mean.feather'
}
train_x = load_feather(train_x, cols_to_rplc_train)

In [25]:
train_x = train_x.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Sex', 'Embarked', 'Pclass'], axis=1)
train_x = train_x.dropna()
train_y = train_y.iloc[train_x.index]

In [26]:
model = ModelKeras(logging=True)
layer1 = Dense(1024, activation='relu', input_shape=(train_x.shape[1],))
layer2 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer4 = Dense(1, activation='sigmoid')
model.add_layers(layer1, layer2, layer3, layer4)
model.compile()
model.set_fit_params(epochs=10, verbose=0)

[INFO] 2021-02-13 19:12:13,303 >>	=== NN KERAS MODEL ===


In [27]:
pred_train = get_pred_result(model, train_x, train_y)
get_acc_and_logloss(pred_train, train_y, logging=False)

data size: 891
correct predictions: 634
accuracy: 0.7115600449
logloss: 0.6090867461
